In [1]:
import numpy as np
import pandas as pd

ca_meta = pd.read_csv('../ca/ca_meta.csv')
sd_meta = ca_meta[ca_meta.District == 11]
sd_meta = sd_meta.reset_index()
sd_meta = sd_meta.drop(columns=['index'])
sd_meta.to_csv('sd_meta.csv', index=False)
print(sd_meta[sd_meta.duplicated(subset=['Lat', 'Lng'])])
sd_meta

Empty DataFrame
Columns: [ID, Lat, Lng, District, County, Fwy, Lanes, Type, Direction, ID2]
Index: []


,ID,Lat,Lng,District,County,Fwy,Lanes,Type,Direction,ID2
0,1114091,32.544463,-117.032486,11,San Diego,I5-N,6,Mainline,N,6931
1,1118333,32.551690,-117.045725,11,San Diego,I5-N,4,Mainline,N,6932
2,1118348,32.558459,-117.061845,11,San Diego,I5-N,4,Mainline,N,6933
3,1114720,32.561334,-117.067081,11,San Diego,I5-N,4,Mainline,N,6934
4,1118352,32.569052,-117.076507,11,San Diego,I5-N,4,Mainline,N,6935
...,...,...,...,...,...,...,...,...,...,...
711,1123276,32.564373,-116.969279,11,San Diego,I905-W,3,Mainline,W,7642
712,1122394,32.564336,-116.961909,11,San Diego,I905-W,3,Mainline,W,7643
713,1123261,32.564112,-116.950744,11,San Diego,I905-W,3,Mainline,W,7644
714,1123256,32.560935,-116.944559,11,San Diego,I905-W,3,Mainline,W,7645


In [2]:
sd_meta_id2 = sd_meta.ID2.values.tolist()
print(len(sd_meta_id2))

ca_rn_adj = np.load('../ca/ca_rn_adj.npy')
print(ca_rn_adj.shape)

sd_rn_adj = ca_rn_adj[sd_meta_id2]
sd_rn_adj = sd_rn_adj[:,sd_meta_id2]
print(sd_rn_adj.shape)

np.save('sd_rn_adj.npy', sd_rn_adj)

716
(8600, 8600)
(716, 716)


In [6]:
year = '2021'  # please specify the year, our experiments use 2019

sd_meta.ID = sd_meta.ID.astype(str)
sd_meta_id = sd_meta.ID.values.tolist()

ca_his = pd.read_hdf('../ca/ca_his_' + year +'.h5')
sd_his = ca_his[sd_meta_id]
data = sd_his['1114091'][:672]

# np.save('7day.npy', data)
print(data)

Time
2021-01-01 00:00:00    39.0
2021-01-01 00:15:00    36.0
2021-01-01 00:30:00    38.0
2021-01-01 00:45:00    43.0
2021-01-01 01:00:00    51.0
                       ... 
2021-01-07 22:45:00    51.0
2021-01-07 23:00:00    51.0
2021-01-07 23:15:00    56.0
2021-01-07 23:30:00    47.0
2021-01-07 23:45:00    41.0
Freq: 15T, Name: 1114091, Length: 672, dtype: float64


In [7]:
sd_his.to_hdf('sd_his_' + year + '.h5', key='t', mode='w')
sd_his.shape

(35040, 716)

In [7]:
# code for checking adj stat
sd_rn_adj = np.load('sd_rn_adj.npy')
node_num = sd_rn_adj.shape[0]

print(sd_rn_adj[0,0])
sd_rn_adj[np.arange(node_num), np.arange(node_num)] = 0
print(sd_rn_adj[0,0])

print('edge number', np.count_nonzero(sd_rn_adj))
print('node degree', np.mean(np.count_nonzero(sd_rn_adj, axis=-1)))
print('sparsity', np.count_nonzero(sd_rn_adj) / (node_num**2) * 100)

1.0
0.0
edge number 17319
node degree 24.18854748603352
sparsity 3.378288755032614
